In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from collections import Counter

from imblearn.over_sampling import SMOTE

from preproc3 import preprocessing
from alg7 import model

In [2]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

PREPROCESSED_DATA='preprocessed_data/'
RESULTS='results/'

In [3]:
train = pd.read_csv(TRAIN) # Cargo datos de entrenamiento
test = pd.read_csv(TEST) # Cargo datos de test

# Eliminamos el campo id ya que no se debe usar para predecir
test_ids = test['id']
del test['id']
del train['id']

# Cambiamos el nombre a la columna Año para poder manejarla correctamente
train.rename(columns = {'Año':'Anio'}, inplace = True)
test.rename(columns = {'Año':'Anio'}, inplace = True)

In [4]:
Counter(train.Precio_cat) # Clases desbalanceadas

Counter({1: 269, 2: 602, 3: 2211, 4: 978, 5: 759})

In [6]:
train, label, test = preprocessing(train, test)
# Guardar los datos ya procesados
np.savez_compressed(PREPROCESSED_DATA+'binScale', train, label, test)

In [7]:
train.shape

(9125, 52)

In [8]:
model

GradientBoostingClassifier(n_estimators=500, random_state=25)

In [9]:
scores=cross_val_score(model, train, label, cv=5)
print(scores)
print(np.mean(scores))

[0.90575342 0.91506849 0.91123288 0.92767123 0.91178082]
0.9143013698630137


## Generar fichero de Kaggle

In [11]:
model.fit(train,label)
# Ahora predecimos
predict = model.predict(test)
predict = list(map(int,predict))
# Generamos 
df_result = pd.DataFrame({'id': test_ids, 'Precio_cat': predict})
df_result.to_csv(RESULTS+"try15.csv", index=False)